### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта

2. при обучении моделей обязательно использовать кроссвалидацию

3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, roc_auc_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from scipy.sparse import hstack
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
title_dict_0 = {'fontsize': 20, 'fontweight': 'bold', 'color': 'DarkRed', 'family': 'Calibri'}
label_dict_0 = {'fontsize': 14, 'color': 'DarkBlue', 'family': 'Calibri'}

In [2]:
data = pd.read_csv('train_case2.csv', sep=';', index_col='id')
data

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
99993,19240,2,168,76.0,120,80,1,1,1,0,1,0
99995,22601,1,158,126.0,140,90,2,2,0,0,1,1
99996,19066,2,183,105.0,180,90,3,1,0,1,0,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


In [4]:
target_name = 'cardio'
target = data[target_name]
data = data.drop(target_name, axis=1)

In [5]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=0)

In [6]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame 
    to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform 
    additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

In [7]:
continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'gluc', 'cholesterol']
base_cols = ['smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))
    
for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))
    
for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

In [8]:
### Теперь объединим все наши трансформеры с помощью FeatureUnion
feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

Добавим классификатор и запустим кросс-валидацию

In [9]:
%%time
classifiers = {
                'LogisticRegression': LogisticRegression(random_state=42),
                'RandomForestClassifier': RandomForestClassifier(random_state=42),
                'Catboost': CatBoostClassifier(random_state=42, silent=True)
              }
scores = dict()

for name, classifier in classifiers.items():
    pipeline = Pipeline([
        ('features', feats),
        ('classifier', classifier),
    ])


    #запустим кросс-валидацию
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=16, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print('CV score is {}+-{}'.format(cv_score, cv_score_std))

    #обучим пайплайн на всем тренировочном датасете
    pipeline.fit(X_train, y_train)
    y_score = pipeline.predict_proba(X_test)[:, 1]
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test.values, y_score)
    scores[name] = dict()
    scores[name]['precision'] = precision[ix]
    scores[name]['recall'] = recall[ix]
    scores[name]['thresholds'] = thresholds[ix]
    scores[name]['fscore'] = fscore[ix]
    scores[name]['roc_auc'] = roc_auc

CV score is 0.7869463146952662+-0.00845027103984613
CV score is 0.7735679290473028+-0.007282985068979065
CV score is 0.8022055901386136+-0.007152833914821671
Wall time: 6min 52s


In [10]:
for key, value in scores.items():
    print(key)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f, ROC-AUC=%.3f, ' % (value['thresholds'], 
                                                                        value['fscore'],
                                                                        value['precision'],
                                                                        value['recall'],
                                                                        value['roc_auc']
                                                                        ))
    print()

LogisticRegression
Best Threshold=0.386124, F-Score=0.730, Precision=0.647, Recall=0.838, ROC-AUC=0.784, 

RandomForestClassifier
Best Threshold=0.323333, F-Score=0.719, Precision=0.632, Recall=0.833, ROC-AUC=0.771, 

Catboost
Best Threshold=0.388484, F-Score=0.739, Precision=0.692, Recall=0.793, ROC-AUC=0.801, 



In [11]:
# согласно заданию привел к таблице:
scores = pd.DataFrame(scores).T
scores

,precision,recall,thresholds,fscore,roc_auc
LogisticRegression,0.647184,0.838134,0.386124,0.730385,0.784207
RandomForestClassifier,0.631795,0.832834,0.323333,0.718517,0.770976
Catboost,0.691967,0.792972,0.388484,0.739035,0.801146


4. сделать выводы о том, какая модель справилась с задачей лучше других

Как обычно Catboost работает "из коробки" уже хорошо, даже с учетом доверительных интервалов.
Логрег и случайный лес тоже имеют значимые отличия, но я почему-то думал, что лес даст лучший результат.
Но мы и параметры не настраивали

5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого). 

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно). 
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

Пока что опущу этот вопрос)